In [1]:
import requests
import re
import os
import time

from urllib import request
from bs4 import BeautifulSoup

import glob
import re
from gensim.models import word2vec

import math

In [2]:
def nocScraping(novelID, limit=0):
   
   #set UA
   header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:62.0) Gecko/20100101 Firefox/62.0"}
   
   #set cookie
   cookie = {"over18": "yes"}
   
   #親URLをセット
   chapterlistUrl = "https://novel18.syosetu.com/" + str(novelID) + "/"
   responseCL  = requests.get(url=chapterlistUrl, headers=header, cookies=cookie)
   chapterlistHtml = responseCL.content
   soupCL = BeautifulSoup(chapterlistHtml, "html.parser")
   sublist = soupCL.find_all("dl", attrs={"class","novel_sublist2"})
   #print(len(sublist))
   
   
   novelDirPass = "data/" + novelID
   count = 1
   
   if(limit == 0):
       countlimit = len(sublist) + 1
   else:
       countlimit = limit + 1
   
   while(count < countlimit):
       #set url
       url = "https://novel18.syosetu.com/" + str(novelID) + "/" + str(count) + "/"

       #get html
       response  = requests.get(url=url, headers=header, cookies=cookie)
       html = response.content

       #set BeautifulSoup
       soup = BeautifulSoup(html, "html.parser")

       #scraping
       mainText = soup.find("div", attrs={"id":"novel_honbun"})
       try:
           mainTextLines = mainText.find_all("p")
       except AttributeError:
           pass

       allText = ""

       for mainTextLine in mainTextLines:
           text = str(mainTextLine)
           text = re.sub("<p.*\">","",text)
           text = re.sub("</p>","",text)
           text = text.replace("<br/>","")
           if(text != ""):
               allText = allText + text + "\n"

       allText = re.sub("\n\n","\n",allText)
       allText = re.sub("　","",allText)
       #print(allText)
       
       os.makedirs(novelDirPass, exist_ok=True)
       newFilePass = novelDirPass + "/" + novelID + "_" + str(count) + ".txt"
       if not os.path.isfile(newFilePass):
           with open(newFilePass, mode="w", encoding="UTF-8") as f:
               f.write(allText)
       
       count = count +1
       
def getNcode(limit):
   response = requests.get("https://api.syosetu.com/novel18api/api",params={"out":"json","nocgenre":1,"sasie":0,"type":"re","ispickup":1,"lim":limit,"of":"n"})
   #print(response.json()[1]["ncode"])
   count = 1
   ncodelist=[]
   
   while(count < limit+1):
       ncodelist.append(response.json()[count]["ncode"])
       count = count + 1
   #print(ncodelist)
   return ncodelist
   

def getNovelText(number,limit=0):
   ncodelist = getNcode(number)
   count = 1
   while(count < number):
       nocScraping(ncodelist[count],limit)
       count = count + 1
       print("count = "+str(count)+" ("+str(math.floor((count/number)*100))+"%)")
       time.sleep(1.5)

In [3]:
getNcode(10) # 関数テスト

['N7831FX',
 'N6999FX',
 'N1339FW',
 'N7377FO',
 'N5341GA',
 'N9297FQ',
 'N1543GB',
 'N7751GF',
 'N7217EI',
 'N0467GA']

In [4]:
getNovelText(500,10)

count = 2 (0.4%)
count = 3 (0.6%)
count = 4 (0.8%)
count = 5 (1.0%)
count = 6 (1.2%)
count = 7 (1.4000000000000001%)
count = 8 (1.6%)
count = 9 (1.7999999999999998%)
count = 10 (2.0%)
count = 11 (2.1999999999999997%)
count = 12 (2.4%)
count = 13 (2.6%)
count = 14 (2.8000000000000003%)
count = 15 (3.0%)
count = 16 (3.2%)
count = 17 (3.4000000000000004%)
count = 18 (3.5999999999999996%)
count = 19 (3.8%)
count = 20 (4.0%)
count = 21 (4.2%)
count = 22 (4.3999999999999995%)
count = 23 (4.6%)
count = 24 (4.8%)
count = 25 (5.0%)
count = 26 (5.2%)
count = 27 (5.4%)
count = 28 (5.6000000000000005%)
count = 29 (5.800000000000001%)
count = 30 (6.0%)
count = 31 (6.2%)
count = 32 (6.4%)
count = 33 (6.6000000000000005%)
count = 34 (6.800000000000001%)
count = 35 (7.000000000000001%)
count = 36 (7.199999999999999%)
count = 37 (7.3999999999999995%)
count = 38 (7.6%)
count = 39 (7.8%)
count = 40 (8.0%)
count = 41 (8.200000000000001%)
count = 42 (8.4%)
count = 43 (8.6%)
count = 44 (8.799999999999999%)


In [5]:
word_list = []

from janome.tokenizer import Tokenizer
t = Tokenizer()
def extract_words(text):
  tokens = t.tokenize(text)
  return [token.base_form for token in tokens if token.part_of_speech.split(",")[0] in ["名詞", "動詞", "形容詞"]]

In [6]:
for path in glob.glob("./data/*/*.txt"):
  with open(path , encoding="UTF-8") as f:
    try:
      for s_line in f:
        s_line = s_line.replace("<ruby><rb>","")
        s_line = s_line.replace("</rb><rp>(</rp><rt>","")
        s_line = s_line.replace("</rt><rp>(</rp></ruby>","")
        word_list.append(extract_words(s_line))
    except UnicodeDecodeError:
      pass

with open("./wordlist.txt", mode="w", encoding="UTF-8") as f:
  f.write(str(word_list))

model = word2vec.Word2Vec(word_list , size=100, min_count=5, window=5, iter=100)
model.save("dosukebe.model")

In [12]:
model.wv.most_similar("オチンチン")

[('オチンポ', 0.7420669794082642),
 ('オマンコ', 0.6633195877075195),
 ('ちんちん', 0.6573238372802734),
 ('ちんぽ', 0.6298401951789856),
 ('チンポ', 0.5934522747993469),
 ('モノ', 0.5559830665588379),
 ('ペニス', 0.554746150970459),
 ('逸物', 0.5484950542449951),
 ('チン', 0.5441779494285583),
 ('巨根', 0.5295102596282959)]